# 모듈 임포트

In [ ]:
!pip install sentence_transformers      
!pip install konlpy                     # 한국어 정보처리를 위한 패키지

     |████████████████████████████████| 79 kB 4.8 MB/s 
     |████████████████████████████████| 3.8 MB 47.1 MB/s 
     |████████████████████████████████| 1.2 MB 48.0 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 6.5 MB 55.2 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
     |████████████████████████████████| 596 kB 79.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=738e7ca6e45ddde2d55cfc8918e21d9d54812c52e88792d15d4119c99bb7c901
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 59.9 MB/s 
     |████████████████████████████████| 448 kB 83.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np  
import itertools    # 효율적인 반복을 위한 모듈

from konlpy.tag import Okt  # 트위터 형태소 분석기를 계승하는 프로젝트
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

path = '/content/drive/Shareddrives/NLP모델링/강의평 데이터/크롤링 통합'
file_list = os.listdir(path)

In [ ]:
# 전체 데이터를 한 데이터프레임으로 합치기
import pandas as pd

df = pd.DataFrame()

for f in file_list:
    data = pd.read_csv(path+ '/' + f)
    df = pd.concat([df, data])

df.shape

(34938, 7)

In [ ]:
# 중복 행 개수 파악
df.duplicated().sum()

1966

In [ ]:
# 중복 행 제거
df = df.reset_index(drop=True)
df = df.drop_duplicates(keep='first')

df.shape    # 34938 - 1966

(32972, 7)

In [ ]:
# 강의명/교수명 컬럼 생성
df['강의명/교수명'] = df['강의명'] + '/' + df['교수명']

df.head()

,학정번호,강의명,교수명,유의사항,별점,수강시기,강의평,강의명/교수명
0,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",4.0,19년 1학기,과제는 자잘하게 많이 내주시지만 큰 시간이 소요된다는 생각을 해본적은 없음. 수업 ...,대학기본영어Ⅰ/전진희
1,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",3.0,19년 1학기,과제가 진짜 일주일에 두번씩 있음\n어려운건 아닌데 나처럼 귀찮아하다가 한두개 안해...,대학기본영어Ⅰ/전진희
2,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",3.0,19년 1학기,잔 과제가 많고 쪽지 시험도 많다. 그리고 되게 꼼꼼하셔서 그냥 숙제로 해 오는 받...,대학기본영어Ⅰ/전진희
3,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",5.0,19년 1학기,프롶 진희전 . 처음엔 되게 깐깐할 것 같았는데 나름의 인간적 면모도 보여주시고 좋...,대학기본영어Ⅰ/전진희
4,YCC1001-01-00,대학기본영어Ⅰ,전진희,"대면강의, 절대평가",3.0,19년 1학기,수업은 간단한데 과제가 매우 많음 과제는 쉬운편\n시험이랑 퀴즈는 교재 외우면 됨\...,대학기본영어Ⅰ/전진희


In [ ]:
# 한 강의당 한 행으로 데이터 합치기
df_copy = df.copy()

review = pd.DataFrame(df_copy.set_index('강의명/교수명').loc[:,'강의평'])
review.columns = ["강의평"]

rating = pd.DataFrame(df.groupby('강의명/교수명').mean()['별점'])
rating.columns = ['평균 별점']

ratingcount = pd.DataFrame(df.groupby('강의명/교수명').count()['별점'])
ratingcount.columns = ['강의평 개수']

In [ ]:
df_X = pd.concat([rating, ratingcount], axis=1)
df_X = df_X.join(review, how='left')

In [ ]:
df_X

,평균 별점,강의평 개수,강의평
강의명/교수명,,,
19세기미국소설/양석원,3.736842,19,다른 강평 말대로 양석원 교수님 수업 중에 가장 널널했다. 과제가 중간 + 기말 보...
19세기미국소설/양석원,3.736842,19,양석원 교수님 강의 중에서 제일 널널하고 좀 쉬운(?) 강의인 듯 합니다. 허클베리...
19세기미국소설/양석원,3.736842,19,나랑 안 맞았다. 성적 기준도 모르겠고 강의 퀄리티도 내 예상보다 별로였다. 약간 ...
19세기미국소설/양석원,3.736842,19,매우 꼼꼼하게 텍스트를 분석해주시고 그 당시 미국사회와 연관시켜서 설명해주십니다\n...
19세기미국소설/양석원,3.736842,19,교수님 말 진짜 빠르신데 텍스트 꼼꼼히 봐주시고 이번에 진짜 시간이 없어서 수업시간...
...,...,...,...
후성유전체학/노재석,4.000000,11,0. 개인적으로 후성유전체학에 관심있었어서 좋은데.....\n1. 중간기말 둘다 치...
후성유전체학/노재석,4.000000,11,시험이 팔이 아픔\n당연히 전부 서술형. 중간 25문 기말 30문. 각 4점.\n하...
후성유전체학/노재석,4.000000,11,최근 발표된 논문이나 화제가 되는 것들을 수업에서 들을 수 있어서 좋음. 다른 학생...


In [ ]:
df_review = pd.concat([rating, ratingcount], axis=1)
df_review["강의평"] = ""

for i in set(df_X.index):
    reviewstr = ''
    for j in range(len(df_X.loc[i])):
        reviewstr += str(df_X.loc[i, '강의평'][j])
    df_review.loc[i, "강의평"] = reviewstr

df_review

,평균 별점,강의평 개수,강의평
강의명/교수명,,,
19세기미국소설/양석원,3.736842,19,다른 강평 말대로 양석원 교수님 수업 중에 가장 널널했다. 과제가 중간 + 기말 보...
20세기영시/김준환,4.583333,12,작가별로 특징을 잘 정리하는게 굉장히 중요한것 같습니다. 수업 집중해서 잘 듣고 필...
20세기중국혁명과사회구성/박경석,4.000000,7,"학생들 발표 주제와 관련된 논문을 미리 읽고 요약해오는 과제가 있는데, 이게 조금은..."
21세기한국의역사적이해/이세영,3.000000,2,밑의 5점짜리 강의평가 보고 수강하려는 학생이 계시면 절대 추천드리지 못해서 글을 ...
2D디지털디자인/김현정,4.636364,22,과제는 좀 힘들었지만 교수님 피드백 꼼꼼하고 솔직하게 너무 잘 해주심ㅜ교수님이 학생...
...,...,...,...
회계원리(1)/이호영,2.816327,49,무난하고 좋은 교수님. 단점이라면 교재 이상의 무언가가 없다는 것. 그래도 독학 열...
회계원리(1)/정수미,4.000000,3,"처음에는 어려웠다고 생각했는데, 과제하면서 교재에 있는 연습문제도 같이 풀어보니까 ..."
회계원리(1)/최원욱,4.608696,92,"과목 특성상 난이도가 있지만, 그래도 학점을 엄청 잘 주시는 듯합니다. 중간 기말 ..."


In [ ]:
df = df_review

# 형태소 분석기로 명사 / 형용사만 추출한 문서 만들기

In [ ]:
# 형태소 분석기로 명사/형용사만 추출한 문서 만들기
okt = Okt()

review_list = df.loc[:, '강의평'].to_list() # 강의평 열만 list로 추출
doc_list = []   # tokenized_doc: 전체 단어에 대한 (단어:품사) 리스트
word_list = []  # tokenized_word: 명사, 형용사만 추출

for i, review in enumerate(review_list):
    tokenized_doc = okt.pos(review)
    tokenized_word = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') or (word[1] == 'Adjective')])
    doc_list.append(tokenized_doc)
    word_list.append(tokenized_word)

In [ ]:
# tokenized_doc
df_doc = pd.DataFrame({"tokenized_doc":doc_list})
df_doc.index = df.index
df_doc.head()

,tokenized_doc
강의명/교수명,
19세기미국소설/양석원,"[(다른, Noun), (강평, Noun), (말, Noun), (대로, Josa)..."
20세기영시/김준환,"[(작가, Noun), (별로, Noun), (특징, Noun), (을, Josa)..."
20세기중국혁명과사회구성/박경석,"[(학생, Noun), (들, Suffix), (발표, Noun), (주제, Nou..."
21세기한국의역사적이해/이세영,"[(밑, Noun), (의, Josa), (5, Number), (점, Noun),..."
2D디지털디자인/김현정,"[(과제, Noun), (는, Josa), (좀, Noun), (힘들었지만, Adj..."


In [ ]:
# tokenized_word
df_word = pd.DataFrame({"tokenized_word":word_list})
df_word.index = df.index
df_word.head()

,tokenized_word
강의명/교수명,
19세기미국소설/양석원,다른 강평 말 양석원 교수 수업 중 가장 널널 과제 중간 말 보고서 점 시험 방식 ...
20세기영시/김준환,작가 별로 특징 정리 굉장히 중요한것 같습니다 수업 집중 필기 좋은 점수 수 있습니...
20세기중국혁명과사회구성/박경석,학생 발표 주제 관련 논문 미리 요약 과제 있는데 이 조금 귀찮았지만 수업 더 유익...
21세기한국의역사적이해/이세영,밑 점 강의 평가 보고 수강 학생 계시 절대 추천 글 역사 수업 살짝 편견 가지 계...
2D디지털디자인/김현정,과제 좀 힘들었지만 교수 피드백 꼼꼼하고 솔직하게 해 주심 교수 학생 아래 강평 저...


# 최빈 단어 파악을 위한 Bag of Word

In [ ]:
# Bag of Word

from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
  # 온점 제거 및 형태소 분석
  document = document.replace('.', '')
  tokenized_document = okt.morphs(document)

  word_to_index = {}
  bow = []

  for word in tokenized_document:  
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index)  
      # BoW에 전부 기본값 1을 넣는다.
      bow.insert(len(word_to_index) - 1, 1)
    else:
      # 재등장하는 단어의 인덱스
      index = word_to_index.get(word)
      # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
      bow[index] = bow[index] + 1

  return word_to_index, bow

In [ ]:
# 전체 강의평에 대한 Bag of Word: 최빈 단어 파악
fullword = ' '.join(word_list)

doc = str(fullword)
vocab, bow = build_bag_of_words(doc)

print('vocabulary :', vocab)
print('bag of words vector :', bow)

from pandas import Series, DataFrame
bow_df = DataFrame({"nouns" : list(vocab.keys()), "bow" : list(bow)})
bow_df = bow_df.sort_values("bow", ascending=False)
bow_df.head(20)

vocabulary : {'다른': 0, '강평': 1, '말': 2, '양석원': 3, '교수': 4, '수업': 5, '중': 6, '가장': 7, '널널': 8, '과제': 9, '중간': 10, '보고서': 11, '점': 12, '시험': 13, '방식': 14, '풀': 15, '주관': 16, '건': 17, '똑같지만': 18, '내용': 19, '자체': 20, '어렵지': 21, '편했다': 22, '가지': 23, '아쉬웠던': 24, '게': 25, '아니라는': 26, '주제가': 27, '자연': 28, '주의': 29, '조금': 30, '뜬금': 31, '없다고': 32, '생각': 33, '강의': 34, '제일': 35, '좀': 36, '쉬운': 37, '듯': 38, '허클베리핀': 39, '같은': 40, '친숙한': 41, '문학': 42, '대해': 43, '공부': 44, '재밌습니다': 45, '뭣': 46, '피드백': 47, '강': 48, '추해요': 49, '나': 50, '안': 51, '성적': 52, '기준': 53, '퀄리티': 54, '내': 55, '예상': 56, '별로': 57, '약간': 58, '고등학교': 59, '느낌': 60, '소통': 61, '그렇고': 62, '뭔가': 63, '그랬다': 64, '반말': 65, '쓰시': 66, '조심': 67, '해': 68, '사람': 69, '만족했나': 70, '염': 71, '좋겠다': 72, '이번': 73, '학기': 74, '마음': 75, '에타': 76, '평': 77, '속': 78, '매우': 79, '꼼꼼하게': 80, '텍스트': 81, '분석': 82, '그': 83, '당시': 84, '미국': 85, '사회': 86, '연관': 87, '설명': 88, '발표': 89, '한번': 90, '있고': 91, '대체': 92, '자기': 93, '주장': 94, '근거': 95, '제시': 96, '줄': 97, '좋은

,nouns,bow
5,수업,32777
4,교수,26824
34,강의,18168
13,시험,16863
118,것,16417
9,과제,13961
111,수,11304
2,말,10725
328,학점,10081
33,생각,8521


# n-gram

In [ ]:
# n-gram 만들기
n_gram_range = (3, 4) # 3, 4개의 단어를 한 묶음으로 간주
candidates_list = []

for i in range(len(word_list)):
    try:      
        count = CountVectorizer(ngram_range=n_gram_range).fit([str(word_list[i])])
        candidates = count.get_feature_names_out()
        candidates_list.append(candidates)
    except:
        candidates_list.append("")

df_candidates = pd.DataFrame({"candidates":candidates_list})
df_candidates.index = df.index

print('trigram 예시 출력 :', candidates_list[0][:5])

trigram 예시 출력 : ['가까이 차지 문제' '가까이 차지 문제 논문' '가끔 귀여우시고 정이' '가끔 귀여우시고 정이 온화하' '가끔 무슨 이야기']


# SBERT

In [ ]:
df.index

Index(['19세기미국소설/양석원', '20세기영시/김준환', '20세기중국혁명과사회구성/박경석', '21세기한국의역사적이해/이세영',
       '2D디지털디자인/김현정', '3D디지털디자인/이예슬', 'ACADEMIC-INDUSTRY PROJECT/오상은',
       'ADVANCED CHEMISTRY & EXPERIMENTS/조현모,엄지원',
       'ADVANCED KOREAN READING/전지은',
       'ADVANCED STUDIES IN FICTION AND PROSE/김성희',
       ...
       '환경재난의과학적이해/우남칠', '환자안전/이승은', '활과리라:생물학과철학의창조적접점찾기/김응빈,김동규',
       '회계원리(1)/기랄안드레스', '회계원리(1)/이한솔', '회계원리(1)/이호영', '회계원리(1)/정수미',
       '회계원리(1)/최원욱', '후생경제학/김세민', '후성유전체학/노재석'],
      dtype='object', name='강의명/교수명', length=1253)

In [ ]:
# SBERT 로드
# 전체 데이터셋에 대해 학습시키는데 시간이 너무 오래 걸려서 원하는 수업만 요약해볼 수 있도록 변경
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

doc_embedding = model.encode([df_word.loc["미분적분학/이명숙", "tokenized_word"]])          # 빈 칸에 "강의명/교수명" 입력 (위 df.index 참고)
candidate_embeddings = model.encode(df_candidates.loc["미분적분학/이명숙", "candidates"])   # 빈 칸에 "강의명/교수명" 입력 (위 df.index 참고)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 문서와 가장 유사한 키워드 5개; 문서를 대표하기 좋은 키워드
top_n = 5

distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [df_candidates.loc["미분적분학/이명숙", "candidates"][index] for index in distances.argsort()[0][-top_n:]]  # 빈 칸에 "강의명/교수명" 입력

print(keywords)

['독학 힘들어요 미적분 내용', '매번 공부 빡치지만 막판', '수학 수월했겠지만 문과 힘들진', '경우 어렵고 많기에 공부', '학생 힘듭니다 많아도 교수']


# SBERT - Max Sum Similarity
- 후보 간의 유사성은 최소화, 후보와 문서의 유사성은 극대화

In [ ]:
_ = df_candidates.loc["미분적분학/이명숙", "candidates"]
_[:150]

array(['가능하지만 권장 진도', '가능하지만 권장 진도 빠르셔서', '가장 어려운 부분', '가장 어려운 부분 공부',
       '가지 멋지게 도전', '가지 멋지게 도전 교수', '가형 통계학 내용', '가형 통계학 내용 미분적분학',
       '감사 하단 생각', '감사 하단 생각 거임', '감자 강의 좋은지는', '감자 강의 좋은지는 열정',
       '갓띵숙 천사 이심', '갓띵숙 천사 이심 수업', '강의 강의 교수', '강의 강의 교수 추천', '강의 경우 문과',
       '강의 경우 문과 듣기', '강의 교수 정말', '강의 교수 정말 열정', '강의 교수 진짜',
       '강의 교수 진짜 같은', '강의 교수 추천', '강의 교수 추천 교수', '강의 기존 단위',
       '강의 기존 단위 높은', '강의 듣기 교수', '강의 듣기 교수 그럼에도', '강의 리다 빠른',
       '강의 리다 빠른 그러니', '강의 문과 강의', '강의 문과 강의 학년', '강의 문과 점수',
       '강의 문과 점수 만족해요', '강의 문제 과목', '강의 문제 과목 자체', '강의 보고 과제',
       '강의 보고 과제 제출', '강의 심화 내용', '강의 심화 내용 노력', '강의 아니었군요 호호',
       '강의 아니었군요 호호 같은', '강의 아님 이과', '강의 아님 이과 의대', '강의 어떻게 학기',
       '강의 어떻게 학기 마치', '강의 엄청나셔서 명강', '강의 엄청나셔서 명강 솔직히', '강의 열의 넘치게',
       '강의 열의 넘치게 아주', '강의 영상 이상인', '강의 영상 이상인 경우', '강의 응용 통계학',
       '강의 응용 통계학 전공', '강의 이번 학기', '강의 이번 학기 교수', '강의 입니다 학기',
       '강의 입니다 학기 동안', '강의 있습니다 이과', '강의 있습니다 이과 경우', '강의 자료 좋고',
       '강의 자료 좋고 교

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
    words_idx = list(distances.argsort()[0][-nr_candidates:])   # argsort(): 작은 값부터 순서대로 데이터의 위치를 반환
                                                                # words_idx에는 문서와의 유사도가 높은 nr_candidates개 단어의 위치가 들어가 있음
    words_vals = [candidates[index] for index in words_idx]     # words_vals에는 words_idx의 위치에 해당하는 단어들이 들어가 있음
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]   # np.ix_: 서로 다른 shape을 가진 배열들을 묶어서 처리

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
# 작은 nr_candidates 값; 키워드들간의 유사도가 높음
candidates = df_candidates.loc["미분적분학/이명숙", "candidates"]
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['어렵고 많기에 공부',
 '있는 수강신청 좋겠습니다 힘들어도',
 '수강신청 좋겠습니다 힘들어도 교수',
 '공부 힘드시다면 버거우실 예습',
 '많은 수업 퀴즈']

In [ ]:
# 큰 nr_candidates 값; 키워드들간의 유사도가 낮음
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=100)

['공부 유익한 강의', '힘들었던 학기 시험', '수월했겠지만 문과 힘들진', '퀴즈 자주 매번 공부', '과목 어려운데 통계학 입문']